# Data Management

# 1 How to Read the data

## 1.1 load Pandas and Numpy: data analysis tools.
    transform the csv to dataframe like excel.

In [ ]:
import pandas as pd
import numpy as np

###  Read the csv file

Set the path and read the csv file

In [ ]:
# filename  ='F:\\Dropbox\\Flight\Data\\cleaned_data\\ready_data_1.csv'

In [ ]:
# data = pd.read_csv(filename)

In [ ]:
# df = data.sample(200)

In [ ]:
#df.to_csv(r"F:\Dropbox\2018\Nankai\sample_df.csv")

We can also read it from web

In [ ]:
url = 'https://raw.githubusercontent.com/jukwan/nankai/master/sample_df.csv'

In [ ]:
data = pd.read_csv(url)

If you need a help,
put question mark.
ex)
data.shape? 

In [ ]:
data.shape?

## Take a quick look about data we load

In [ ]:
data.shape 
# This shows the # of (rows, columns)

In [ ]:
data.head() # shows first 5 rows

In [ ]:
data.tail()  # shows last 5 rows

In [ ]:
data.sample(5) # randumly shows the data numbers in parenthesis indicate number of obs to show.

### Check the variables in data frame

In [ ]:
data.columns.values

### Select necessary variables

In [ ]:
 keyval=['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
           'UniqueCarrier', 'AirlineID', 'Carrier', 'TailNum',
            'FlightNum', 'Origin', 'OriginCityName', 
            'Dest', 'DestCityName',  
            'DepTime', 'DepDelay', 'DepDel15',
            'TaxiOut', 'TaxiIn',
            'ArrTime', 'ArrDelay', 'Cancelled',
            'Diverted', 
            'AirTime', 'Flights', 'Distance', 'DistanceGroup',
            'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
            'vis','temp','precip','winter','WindSpeed','WeatherType'
        ]

In [ ]:
df= data[keyval] # make new dataframe which is subset of previous dataset.

In [ ]:
df.head()

Can access columns by name

In [ ]:
df['Dest'].head(4)

In [ ]:
df.describe()

In [ ]:
df['TaxiOut'].describe() # shows basic stats for 1 variable

In [ ]:
df[['TaxiOut','TaxiIn']].describe() # more than one.

### Check the missing values

In [ ]:
df.isnull().sum() 

### Counting unique values in specific variables

In [ ]:
df.Cancelled.value_counts()

In [ ]:
df['Diverted'].value_counts()

In [ ]:
# Create a variable by manipulation
df['winds'] =df['WindSpeed']*1.609 
df.head()

In [ ]:
df['Carrier'].unique()

## 2. Sort the data
    Sort by multiple column values

In [ ]:
df.sort_values(by =['Carrier','DayOfWeek','Origin','Dest'],  ascending=True)

In [ ]:
df.sort_values(by='DepDelay', ascending=False, na_position='first')

### Group by 

In [ ]:
df.groupby(['Origin'])['ArrDelay'].describe()

In [ ]:
df.groupby(['Carrier'])['DepDelay','ArrDelay'].mean()

## Mean can replaced by those functions
# Function	Description
# mean()	Compute mean of groups
# sum()	Compute sum of group values
# size()	Compute group sizes
# count()	Compute count of group
# std()	Standard deviation of groups
# var()	Compute variance of groups
# sem()	Standard error of the mean of groups
# describe()	Generates descriptive statistics
# first()	Compute first of group values
# last()	Compute last of group values
# nth()	Take nth value, or a subset if n is a list
# min()	Compute min of group values
# max()	Compute max of group values

In [ ]:
df.groupby(['Carrier']).get_group('DL').mean()

In [ ]:
a= df.groupby(['Origin','Dest'])['DepDelay','ArrDelay']


In [ ]:
a.mean()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
df['Month'].value_counts().sort_index().plot()
plt.title('Montyly Overall Operations')

In [ ]:
df['Carrier'].value_counts().head(5).sort_index().plot()

plt.xlabel("Carrier",fontsize=10)

In [ ]:
delay_dep= df.groupby(['Carrier','DepDel15'])

delay_dep.size().unstack().head(5).plot()



In [ ]:
delay_dep.size()

# 3. Variable Management

### Drop the variables

In [ ]:
data.drop(['Div1WheelsOn'], axis=1)

### Drop the row by column values

In [ ]:
df.query("Origin =='BOS' or Origin =='ATL'")

In [ ]:
df[df['DepDelay']>0]

In [ ]:
df.query('DepDelay > 0 and ArrDelay ==0')

### Make indicating variable from string variables

In [ ]:
## Snow if weather type contain any SN 
mask =df['WeatherType'].str.contains("SN|FZ", na=False)
                              
df['snow'] =mask.astype(int)
df['snow'].value_counts()

### Creating Dummy Variables

In [ ]:
pd.get_dummies(df, columns=["Carrier"])

In [ ]:
s = pd.get_dummies(df.Carrier)

In [ ]:
df=df.join(s)

In [ ]:
df.head()

In [ ]:
## give id to airport
df['airs'] = df['Carrier'].astype('category')
df['airs'].dtypes
df['airs_cat']= df['airs'].cat.codes
df['airs_cat'].value_counts()

### give market id 
market combination of departure and arrival airport

In [ ]:
df['marketId']=df.groupby(['Origin','Dest']).ngroup().add(1)
df.groupby(['Origin','Dest'])['DepDelay','ArrDelay','marketId'].mean()

## Regression

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
df.columns.values

In [ ]:
#Regression

$ Delay = \beta_1 X_i +\beta_2 Snow$

In [ ]:
# OLS

$ Delay = \beta_1 X_i +\beta_2 Snow$

In [ ]:
Xs= "temp+vis+Distance+winds"
reg= smf.ols('DepDelay ~ +C(Carrier) +' +Xs, data=df).fit()
print (reg.summary())

In [ ]:
Xs=['temp','vis']

In [ ]:
sm_lgt = smf.Logit(df['Cancelled'],df[Xs])

In [ ]:
sm_logit=sm_lgt.fit()

In [ ]:
print(sm_logit.summary())